In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
from sklearn import preprocessing, cluster
import scipy
import scipy.cluster
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from shapely.ops import unary_union
import calendar
from datetime import datetime
import base64

/home/rajat/anaconda3/envs/gisenv/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [3]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [34]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = """with base as (    
    select
            shipping_pincode
        ,   sum(ofd) as ofd
        ,   sum(efd) as efd
    from public.pincode_level_metrics
    where 1=1
    and datetime = date_trunc('day',now()+interval '5.5 hours') - interval '1 day'
    and (efd is not null or efd != 0)
    group by 1
)
,
final as (
    select
        shipping_pincode
    ,   coalesce(ofd::double precision/nullif(efd, 0), 0)*100 as "ofd%"
    from base
    order by 1
    )
    
select * from final"""


# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
     shipping_pincode        ofd%
0              110003   88.888889
1              110005   97.029703
2              110007   97.674419
3              110008   96.992481
4              110009   93.717277
..                ...         ...
421            560109  100.000000
422            560110  100.000000
423            560111  100.000000
424            560114  100.000000
425            800016  100.000000

[426 rows x 2 columns]


In [35]:
# Reading GeoJSON
gdf_geojson = gpd.read_file("/home/rajat/GIS Blitznow/India_Pincodes/india_pincodes.shp")

# If the current CRS is geographic, re-project to UTM (EPSG:32644)
if gdf_geojson.crs.is_geographic:
    gdf_geojson = gdf_geojson.to_crs('EPSG:32644')

In [36]:
# Calculating centroids of pincodes
gdf_geojson = gdf_geojson.to_crs(epsg=4326)

gdf_geojson['latitude'] = gdf_geojson['geometry'].centroid.y
gdf_geojson['longitude'] = gdf_geojson['geometry'].centroid.x

# Ensure the pincode column datatype is consistent
df['shipping_pincode'] = df['shipping_pincode'].astype(str)
gdf_geojson['pincode'] = gdf_geojson['pincode'].astype(str)

# Merge GeoDataFrame and DataFrame
merged_gdf = gdf_geojson.merge(df, left_on='pincode', right_on='shipping_pincode')

In [37]:
# merged_gdf = merged_gdf[merged_gdf['district'] == 'Bangalore']

In [39]:
# Initialize Dash app
app = Dash(__name__)

# Get unique districts for the dropdown options and add 'All Districts' option
districts = merged_gdf['district'].unique()
district_options = [{'label': 'All Districts', 'value': 'All'}] + [{'label': district, 'value': district} for district in districts]

# Define app layout
app.layout = html.Div([
    html.H1("Orders Delivered Yesterday by Pincode", style={'text-align': 'center'}),
    dcc.Dropdown(
        id='district-dropdown',
        options=district_options,
        value='All',  # Default to 'All Districts'
        style={'width': '50%'}
    ),
    dcc.Graph(id='map', figure={}),
])

# Callback to update the graph
@app.callback(
    Output(component_id='map', component_property='figure'),
    Input(component_id='district-dropdown', component_property='value')
)
def update_graph(selected_district):
    if selected_district == 'All':
        filtered_gdf = merged_gdf
    else:
        filtered_gdf = merged_gdf[merged_gdf['district'] == selected_district]

    fig = px.choropleth_mapbox(
        filtered_gdf,
        geojson=filtered_gdf.geometry.__geo_interface__,
        locations=filtered_gdf.index,
        color='ofd%',
        hover_data=['shipping_pincode', 'ofd%'],
        mapbox_style="carto-positron",
        center={"lat": filtered_gdf['latitude'].mean(), "lon": filtered_gdf['longitude'].mean()},
        zoom=10,
        opacity=0.9,
        template='plotly_dark',
        color_continuous_scale="RdYlGn"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8056)